In [ ]:
# Load all libraries
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load all sklearn libraries
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

#import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow.compat.v2 as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from __future__ import absolute_import, division, print_function, unicode_literals
from collections import Counter
#Enable eager execution
#tf.enable_v2_behavior()

In [ ]:
# Import the libraries:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Fix the seeds to reproduce the results:
SEED = 101
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# import training dataset:
train = pd.read_csv("../input/digit-recognizer/train.csv")
train

In [ ]:
# separate the label and features:
X =  train.drop(['label'], 1).values
y = train['label'].values

In [ ]:
# normalize the training set:
X = X/ 255.0


In [ ]:
# print the shape of features:
print(X.shape)


In [ ]:
# load dataset as train and test:
#(train_X, y_train), (test_X, y_test) = mnist.load_data()
# reshape dataset to have a single channel
X = X.reshape(-1, 28, 28, 1)


In [ ]:
from sklearn.model_selection import train_test_split
# Split the train and the validation set for the fitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# print the shape of the train feature(image) and train label(digits)
print(f'train_images.shape = {X_train.shape}; train_labels.shape = {y_train.shape}')

In [ ]:
# print the shape of the test feature(image) and test label(digits)
print(f'test_images.shape = {X_test.shape}')

In [ ]:
# Display the first 20 images
fig, ax = plt.subplots(4, 5, figsize = (10, 8))   # Create a 5x4 plot
ax = ax.ravel()                                   # Flatten the array (helps with the for loop)

# Use a for loop to get the first 20, adding each as a plot
for idx in range(20) :
    ax[idx].imshow(X_train[idx,:,:], cmap=plt.get_cmap('gray'))   # The image
    ax[idx].set_title(y_train[idx])                                  # The value
    ax[idx].axes.xaxis.set_ticks([])                                 # Remove x axis
    ax[idx].axes.yaxis.set_ticks([])                                 # Remove y axis
    
plt.tight_layout()
plt.show()

**Building the Convolutional Neural Network (CNN) model using TensorFlow:**

In [ ]:
model = models.Sequential()
# Get the image size using codes instead of using magic number (32,32,3)
#input_shape = tf.reshape(X_train, [-1, 28, 28, 3])
input_shape =X_train.shape
print(f'The input shape = {input_shape[1:]}')
# The first Convolution layer (input layer) has 32 filters, with the output has 32 channels. The size of the filter is 3*3
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape= input_shape[1:]))
model.add(BatchNormalization())
# The first Convolution layer has 32 filters, with the output has 32 channels. The size of the filter is 3*3
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
# Followed by 2*2 pooling window 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
# The second Convolution layer has 64 filters, with the output has 64 channels. The size of the filter is 3*3
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
# Followed by 2*2 pooling window 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Dropout(0.25))
# The second Convolution layer has 64 filters, with the output has 64 channels. The size of the filter is 3*3
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dropout(0.5))
# The Third Convolution layer has 64 filters, with the output has 64 channels. The size of the filter is 2*2
model.add(layers.Conv2D(512, (2, 2), activation='relu'))
model.add(BatchNormalization()) 
model.add(layers.Dropout(0.5))

**Summarizing the model structures as follows:**

In [ ]:
model.summary()

The height and width of the image decrease from (26,26) to (13,13), ... (2,2).

But we increase the number of filters/channels in the Conv2d operation

The pooling method doesn't change the number of channels, and it only shrinks the image size.

**Since the input is a 4D matrix therefore we need to flattens the input from a 4D to a 2D matrix. There are ten unique cases in the label, so the last layer must use ten neurons.**

In [ ]:
# Convert  feature matrix from 4D to 2D
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(BatchNormalization())
# Add dropout layer
model.add(layers.Dropout(0.5))
# Last layer must have 10 neuron to match the label
model.add(layers.Dense(10))


**Summarizing the model architecture. The output shape must be converted from 4D to 2D.**

In [ ]:
model.summary()

**Plotting both the input and output layer as follows:**

In [ ]:
dot_img_file = 'CNNmodel.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

**Configure the Model:**

In [ ]:
# It is a multiclass classification problem; the label is encoded as an integer.
# Therefore the loss function should be tf.keras.losses.SparseCategoricalCrossentropy:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Train the Model:**

In [ ]:
# Train the model without early stopping (Stop training when a monitored metric has stopped improving):

#callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 3)
history = model.fit(X_train, y_train, epochs=20, 
                    validation_data=(X_test, y_test), verbose = 1)

**Evaluate the model by checking the in-sample fit and out of sample fit:**

In [ ]:
## Evaluate the Model
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
train_history = pd.DataFrame(history.history)
train_history['epoch'] = history.epoch
# Plot train loss
sns.lineplot(x='epoch', y ='loss', data =train_history)
# Plot validation loss
sns.lineplot(x='epoch', y ='val_loss', data =train_history)
# Add legends
plt.legend(labels=['train_loss', 'test_loss'])

In [ ]:
plt.figure(figsize=(8, 8))
# Plot training accuracy
sns.lineplot(x='epoch', y ='accuracy', data =train_history)
# Plot validation accuracy
sns.lineplot(x='epoch', y ='val_accuracy', data =train_history)
# Add legends
plt.legend(labels=['train_accuracy', 'test_accuracy'])

The above output shows there is a rapid increase in train_accuracy from epoch 0 to 1 and then there is a steady increase in accuracy from epoch 1 to 17. Test_accuracy also increases steadily from epoch 1 to 17. There is still room for improvement in test accuracy.

#### Compute the loss and accuracy for the test dataset as follows:

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=0)
print(f' The test loss ={test_loss:.2f} and test accuracy = {test_acc:.2f}')

In [ ]:
# Predict the test datadet:
y_pred = (model.predict(X_test))
Y_predict = np.argmax(y_pred, axis=1)


In [ ]:
# Print the classification report
print(classification_report(y_test, Y_predict))

In [ ]:
# actual and predicted values:
Y_label = pd.DataFrame(y_test)
Y_Predict = pd.DataFrame(Y_predict)
predictions = pd.concat([Y_label, Y_Predict], axis = 1)
predictions.columns = ['Label', 'Prediction']
predictions

In [ ]:
# selected rows with correct predictions:
wrong_pred = predictions.loc[(predictions['Label'] != predictions['Prediction'],['Label', 'Prediction'])]
wrong_pred


In [ ]:
# Display mis-matched items, up to 40 of them
mismatch = np.where(y_test != Y_predict )
num_mismatch = len(mismatch[0])
print("Number of mismatches: ", num_mismatch)

# Set up the number of rows for output
rows = min(5, num_mismatch//5 + 1)


fig, ax = plt.subplots(rows, 5, figsize = (12, rows * 20/8))
ax = ax.ravel()

for i in range(min(len(mismatch[0]), 25)) :
    idx = mismatch[0][i]
    
    ax[i].imshow(np.reshape(X_test[idx], (28, 28)), cmap=plt.get_cmap('gray'))  # The image
    ax[i].set_title("True: {}, Pred: {}".format(y_test[idx], Y_predict [idx]))      # The value & prediction
    ax[i].axes.xaxis.set_ticks([])                                              # Remove x axis
    ax[i].axes.yaxis.set_ticks([])                                              # Remove y axis

if ((num_mismatch < 27) & (num_mismatch % 5 > 0)):
    for i2 in range(i, 5 * rows) :
        # Remove everything for the remaining plots
        ax[i2].axis('off')
        

plt.tight_layout()
plt.show()

In [ ]:
# confusion matrix with actual values (index values) and column names as the predicted values:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, Y_predict)

cm_df = pd.DataFrame(cm)
cm_df

In [ ]:
# read the test dataset as follows:
test = pd.read_csv("../input/digit-recognizer/test.csv")
test

In [ ]:
# change the test set into a list:
test_X =  test.values

In [ ]:
# normalize the test set:
test_X = test_X/ 255.0

In [ ]:
# shape of the test set:
test_X.shape

In [ ]:
# flatten the array as follows:
test_X = test_X.reshape(-1, 28, 28, 1)

In [ ]:
# now the shape of the test set:
test_X.shape

In [ ]:
# Predict the test datadet:
pred = (model.predict(test_X))
predict = np.argmax(pred, axis=1)


In [ ]:
Predict = pd.DataFrame(predict)


In [ ]:
x= range(1,28001)
Predict['Imageid']=pd.DataFrame(x)
Predict.columns = ['Label', 'Imageid']
Predict

In [ ]:
Predict.to_csv('submission.csv', index=False)

The accuracy for the test set is 99.36%. 